# Wrangle that Data

<ul>
    <li><a href="#Gather">Gather</a></li>
    <li><a href="#Assess">Assess</a></li>
    <li><a href="#Clean">Clean</a></li>
</ul>

In [111]:
import pandas as pd
import numpy as np
import requests
import json
import tweepy
import os.path
import json
import time
from glob import glob

## Gather

In [24]:
# load secrets
with open('secrets.json') as secrets_file:
    secrets = json.load(secrets_file)

In [3]:
tweets_raw = pd.read_csv('twitter-archive-enhanced.csv')
tweets_raw.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [15]:
image_predictions_file_name = 'image-predictions.tsv'
image_predictions_source_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [12]:
# make sure image predictions are downloaded
if not os.path.exists(image_predictions_file_name):
    response = requests.get(image_predictions_source_url, stream=True)
    with open(image_predictions_file_name, mode='w', encoding='utf-8') as dest_file:
        for chunk in response.iter_content(decode_unicode=True):
            # filter out keep-alive new chunks
            if chunk:
                dest_file.write(chunk)

In [14]:
image_predictions_raw = pd.read_csv(image_predictions_file_name, sep='\t')
image_predictions_raw.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [62]:
# get twitter API
consumer_key = secrets['twitter']['consumerApiKey']
consumer_secret = secrets['twitter']['consumerSecret']
access_token = secrets['twitter']['accessToken']
access_secret = secrets['twitter']['accessTokenSecret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

twitter_api = tweepy.API(
    auth,
    wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

In [101]:
status_ids = (tweets_raw['expanded_urls']
    .str.extract(r'^http[s]?://twitter\.com/dog_rates/status/(\d+)', expand=False)
    .dropna()
    .drop_duplicates()
    .astype(str))
status_count = len(status_ids)
len(status_ids)

1967

In [104]:
already_downloaded_statuses = (
    pd.Series(os.listdir('extended-statuses'))
        .str.extract(r'^(\d+)\.json$', expand=False)
        .dropna())
len(already_downloaded_statuses)

0

In [109]:
statuses_to_download = set(status_ids) - set(already_downloaded_statuses)
len(statuses_to_download)

425

In [114]:
print('Pulling twitter statuses.')
i = 0
for status_id in statuses_to_download:
    try:
        status = api.get_status(status_id, tweet_mode='extended')
        with open(f'extended-statuses/{status_id}.json', 'w') as target_file:
            json.dump(status._json, target_file, indent=2)
    except tweepy.RateLimitError as rle:
        print(rle)
        time.sleep(60 * 5)
        status = api.get_status(status_id, tweet_mode='extended')
        with open(f'extended-statuses/{status_id}.json', 'w') as target_file:
            json.dump(status._json, target_file, indent=2)
    except Exception as e:
        print(e)
    i += 1
    if i % 100 == 0:
        print(f'Statuses pulled so far: {i}.')
        time.sleep(60)
print('Done.')

Pulling twitter statuses.
Statuses pulled so far: 100.
Statuses pulled so far: 200.
Statuses pulled so far: 300.
Statuses pulled so far: 400.
Done.


In [115]:
already_downloaded_statuses = (
    pd.Series(os.listdir('extended-statuses'))
        .str.extract(r'^(\d+)\.json$', expand=False)
        .dropna())
len(already_downloaded_statuses)

1967

In [122]:
extended_tweets_arr = []
users_arr = []
for file_path in glob('extended-statuses/*.json'):
    with open(file_path, 'r', encoding='utf-8') as status_file:
        status = json.load(status_file)
    extended_tweets_arr.append({
        'id': status['id'],
        'full_text': status['full_text'],
        'created_at': status['created_at'],
        'source': status['source'],
        'in_reply_to_status_id': status['in_reply_to_status_id'],
        'in_reply_to_user_id': status['in_reply_to_user_id'],
        'retweet_count': status['retweet_count'],
        'favorite_count': status['favorite_count'],
        'user_id': status['user']['id']})
    users_arr.append({
        'id': status['user']['id'],
        'followers_count': status['user']['followers_count'],
        'friends_count': status['user']['friends_count'],
        'listed_count': status['user']['listed_count'],
        'favourites_count': status['user']['favourites_count'],
        'created_at': status['user']['created_at'],
        'statuses_count': status['user']['statuses_count']})
extended_tweets_raw = pd.DataFrame(statuses_arr)
users_raw = pd.DataFrame(users_arr)

In [123]:
extended_tweets_raw.head()

,created_at,favorite_count,full_text,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,source,user_id
0,Wed Dec 09 16:52:27 +0000 2015,1563,Rare submerged pup here. Holds breath for a lo...,674632714662858753,NaN,NaN,614,"<a href=""http://twitter.com/download/iphone"" r...",4196983835
1,Mon Dec 21 03:12:08 +0000 2015,2965,This is Tug. He's not required to wear the con...,678774928607469569,NaN,NaN,1009,"<a href=""http://twitter.com/download/iphone"" r...",4196983835
2,Tue Jul 11 00:00:02 +0000 2017,24293,This is Kevin. He's just so happy. 13/10 what ...,884562892145688576,NaN,NaN,4901,"<a href=""http://twitter.com/download/iphone"" r...",4196983835
3,Sat Nov 28 21:34:09 +0000 2015,1264,*screams for a little bit and then crumples to...,670717338665226240,NaN,NaN,523,"<a href=""http://twitter.com/download/iphone"" r...",4196983835
4,Sun Mar 26 01:38:00 +0000 2017,31014,We usually don't rate polar bears but this one...,845812042753855489,NaN,NaN,9504,"<a href=""http://twitter.com/download/iphone"" r...",4196983835


In [121]:
users_raw.head()

,created_at,favourites_count,followers_count,friends_count,id,listed_count,statuses_count
0,Sun Nov 15 21:41:29 +0000 2015,134004,6889373,8,4196983835,4389,7100
1,Sun Nov 15 21:41:29 +0000 2015,134004,6889502,8,4196983835,4475,7100
2,Sun Nov 15 21:41:29 +0000 2015,134004,6889503,8,4196983835,4477,7100
3,Sun Nov 15 21:41:29 +0000 2015,134004,6889443,8,4196983835,4414,7100
4,Sun Nov 15 21:41:29 +0000 2015,134004,6889431,8,4196983835,4413,7100


## Assess